> Projeto Desenvolve <br>
Programação Intermediária com Python <br>
Profa. Camila Laranjeira (mila@projetodesenvolve.com.br) <br>

# 3.14 - ORM

## Exercícios

#### Q1. Conhecendo os dados
Baixe o seguinte csv onde iremos trabalhar. Ele contém informações sobre salários de profissionais de dados de uma empresa hipotética entre 2009 e 2016
* https://github.com/camilalaranjeira/python-intermediario/blob/main/salaries.csv

Suas colunas, descritas na [página do Kaggle que contém o dataset](https://www.kaggle.com/datasets/krishujeniya/salary-prediction-of-data-professions?resource=download), são:
* FIRST NAME: Primeiro nome do profissional de dados (String)
* LAST NAME: Sobrenome do profissional de dados (String)
* SEX: Gênero do profissional de dados (String: 'F' para Feminino, 'M' para Masculino)
* DOJ (Date of Joining): A data em que o profissional de dados ingressou na empresa (Data no formato MM/DD/AAAA)
* CURRENT DATE: A data atual ou a data de referência dos dados (Data no formato MM/DD/AAAA)
* DESIGNATION: O cargo ou designação do profissional de dados (String: ex., Analista, Analista Sênior, Gerente)
* AGE: Idade do profissional de dados (Integer)
* SALARY: Salário anual do profissional de dados (Float)
* UNIT: Unidade de negócios ou departamento em que o profissional de dados trabalha (String: ex., TI, Finanças, Marketing)
* LEAVES USED: Número de licenças utilizadas pelo profissional de dados (Integer)
* LEAVES REMAINING: Número de licenças restantes para o profissional de dados (Integer)
* RATINGS: Avaliações de desempenho do profissional de dados (Float)
* PAST EXP: Experiência de trabalho anterior em anos antes de ingressar na empresa atual (Float)

Na célula a seguir, **carregue os dados do CSV e dê uma olhada neles antes de seguir**.

In [52]:
import pandas as pd
from google.colab import files

# Solicitar o upload do arquivo salaries.csv
print("Por favor, faça o upload do arquivo 'salaries.csv':")
uploaded = files.upload()

# Verificar se o arquivo foi carregado corretamente
if 'salaries.csv' in uploaded:
    # Carregar o dataset
    df = pd.read_csv('salaries.csv')

    # Exibir as primeiras linhas do dataset
    print("\n=== Primeiras 5 linhas do dataset ===")
    display(df.head())

    # Exibir informações gerais sobre

Por favor, faça o upload do arquivo 'salaries.csv':


Saving salaries.csv to salaries (4).csv


#### Q2. Modelando os dados
Você deve **criar um ORM com SQLAlchemy capaz de comportar os dados dessa base**.

* Crie um campo de chave primária `ID`, que deve ser incrementado automaticamente
* Os campos SEX, DESIGNATION e UNIT devem ser definidos como classes `Enum` com os possíveis valores (consulte os valores únicos dessas colunas)
* Para os outros campos, consulte os tipos de dados informados na descrição acima

In [54]:
from sqlalchemy import create_engine, Column, Integer, String, Float, Date, Enum
from sqlalchemy.orm import declarative_base

# Criar a base declarativa
Base = declarative_base()

# Definir os valores únicos para SEX, DESIGNATION e UNIT
# Esses valores serão usados para criar Enums no SQLAlchemy
sex_values = ['F', 'M']  # Valores possíveis para o campo SEX
designation_values = ['Analyst', 'Senior Analyst', 'Manager']  # Exemplo de valores únicos (ajuste conforme necessário)
unit_values = ['IT', 'Finance', 'Marketing']  # Exemplo de valores únicos (ajuste conforme necessário)

# Criar a classe Salaries
class Salaries(Base):
    __tablename__ = 'salaries'

    # Chave primária autoincrementada
    id = Column(Integer, primary_key=True, autoincrement=True)

    # Campos do dataset
    first_name = Column(String)  # Primeiro nome
    last_name = Column(String)   # Sobrenome
    sex = Column(Enum(*sex_values, name='sex_enum'))  # Gênero ('F' ou 'M')
    doj = Column(Date)           # Data de ingresso na empresa
    current_date = Column(Date)  # Data atual/referência
    designation = Column(Enum(*designation_values, name='designation_enum'))  # Cargo
    age = Column(Integer)        # Idade
    salary = Column(Float)       # Salário anual
    unit = Column(Enum(*unit_values, name='unit_enum'))  # Unidade de negócios
    leaves_used = Column(Integer)  # Licenças utilizadas
    leaves_remaining = Column(Integer)  # Licenças restantes
    ratings = Column(Float)      # Avaliações de desempenho
    past_exp = Column(Float)     # Experiência anterior em anos

#### Q3. Estabelecendo uma conexão

Usando o método `create_engine` do SQLAlchemy, crie uma conexão com um novo banco de dados SQLite chamado `salarios`.

In [56]:
# Criar uma conexão com o banco de dados SQLite
engine = create_engine('sqlite:///salarios.db', echo=True)

# Criar uma sessão
Session = sessionmaker(bind=engine)
session = Session()

#### Q4. Criando as tabelas
Crie as tabelas da questão Q2 no banco `salarios`.

In [57]:
# Criar as tabelas no banco de dados
Base.metadata.create_all(engine)

2025-03-28 13:27:56,984 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2025-03-28 13:27:56,986 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("salaries")


INFO:sqlalchemy.engine.Engine:PRAGMA main.table_info("salaries")


2025-03-28 13:27:56,987 INFO sqlalchemy.engine.Engine [raw sql] ()


INFO:sqlalchemy.engine.Engine:[raw sql] ()


2025-03-28 13:27:56,993 INFO sqlalchemy.engine.Engine COMMIT


INFO:sqlalchemy.engine.Engine:COMMIT


#### Q5. Populando

Usando o método `to_sql` da biblioteca Pandas (veja [a documentação](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_sql.html)), popule o banco `salarios` com os dados do csv que você carregou na questão Q1.
* Lembre-se de definir o parâmetro `if_exists='append'` para que as tabelas não sejam dropadas e recriadas.

In [59]:
import pandas as pd
from sqlalchemy import create_engine

# Certifique-se de que o DataFrame `df` já foi carregado anteriormente
# Caso contrário, descomente a linha abaixo para carregar o CSV localmente:
# df = pd.read_csv('salaries.csv')

# Renomear as colunas do DataFrame para corresponder ao modelo ORM
df.rename(columns={
    "FIRST NAME": "first_name",
    "LAST NAME": "last_name",
    "SEX": "sex",
    "DOJ": "doj",
    "CURRENT DATE": "current_date",
    "DESIGNATION": "designation",
    "AGE": "age",
    "SALARY": "salary",
    "UNIT": "unit",
    "LEAVES USED": "leaves_used",
    "LEAVES REMAINING": "leaves_remaining",
    "RATINGS": "ratings",
    "PAST EXP": "past_exp"
}, inplace=True)

# Criar uma conexão com o banco de dados SQLite
engine = create_engine('sqlite:///salarios.db')

# Popular o banco de dados com os dados do DataFrame
df.to_sql('salaries', con=engine, if_exists='append', index=False)

print("Dados populados com sucesso no banco de dados 'salarios.db'.")

Dados populados com sucesso no banco de dados 'salarios.db'.


#### Q6. Consultas SQL vs ORM

Agrupe os dados por DESIGNATION e selecione o mínimo, máximo e a média dos salários (SALARY) divididos por 12. Já que o atributo SALARY é anual, dividir por 12 nos mostrará os valores mensais.

Assumindo que a variável que armazena a sua conexão se chama `engine`, você deve realizar a query acima de três formas:
* Executando a query SQL através de uma instância de conexão retornada pelo método `engine.connect()`
* Executando a query SQL com o método `read_sql_query` do Pandas (veja [a documentação](https://pandas.pydata.org/docs/reference/api/pandas.read_sql_query.html)). Você usará mesma instância `engine.connect()` como um dos parâmetros.
* Executando uma query criada com o módulo `select` do SQLAlchemy. Sua execução deve ser feita através de um objeto `Session` do módulo `orm` do SQLAlchemy (`Session(engine)`).


In [78]:
from sqlalchemy import create_engine, text

# Criar uma conexão com o banco de dados
engine = create_engine('sqlite:///salarios.db')

# Consulta SQL para agrupar os dados por DESIGNATION e calcular as estatísticas salariais
query = """
    SELECT
        DESIGNATION,
        MIN(SALARY / 12) AS min_salary,
        MAX(SALARY / 12) AS max_salary,
        AVG(SALARY / 12) AS avg_salary
    FROM salaries
    GROUP BY DESIGNATION
"""

# Executar a consulta
with engine.connect() as connection:
    result = connection.execute(text(query))  # Use text() para converter a consulta em um objeto executável

    # Exibir os resultados
    for row in result:
        print(row)

('Analyst', 3333.4166666666665, 4165.0, 3751.6759876859924)
('Associate', 5846.166666666667, 8300.25, 7266.915094339626)
('Director', 17832.25, 32342.666666666668, 23914.26562500001)
('Manager', 8343.666666666666, 12407.5, 10522.716049382716)
('Senior Analyst', 4170.333333333333, 5830.5, 4991.778792134825)
('Senior Manager', 12614.416666666666, 16631.416666666668, 14888.689516129034)


In [82]:
# Executar a query SQL usando Pandas
query = """
    SELECT DESIGNATION,
           MIN(SALARY / 12) AS min_salary,
           MAX(SALARY / 12) AS max_salary,
           AVG(SALARY / 12) AS avg_salary
    FROM salaries
    GROUP BY DESIGNATION
"""
result_df = pd.read_sql_query(query, con=engine)

# Exibir os resultados
display(result_df)

,designation,min_salary,max_salary,avg_salary
0,Analyst,3333.416667,4165.000000,3751.675988
1,Associate,5846.166667,8300.250000,7266.915094
2,Director,17832.250000,32342.666667,23914.265625
3,Manager,8343.666667,12407.500000,10522.716049
4,Senior Analyst,4170.333333,5830.500000,4991.778792
5,Senior Manager,12614.416667,16631.416667,14888.689516


In [83]:
# Importar bibliotecas necessárias
from sqlalchemy import create_engine, Column, Integer, String, Float, Date, Enum
from sqlalchemy.orm import declarative_base, Session
from sqlalchemy import select, func

# Definir a base declarativa
Base = declarative_base()

# Definir o modelo Salaries
class Salaries(Base):
    __tablename__ = 'salaries'

    id = Column(Integer, primary_key=True, autoincrement=True)
    first_name = Column(String)
    last_name = Column(String)
    sex = Column(Enum('F', 'M', name='sex_enum'))
    doj = Column(Date)
    current_date = Column(Date)
    designation = Column(String)  # Cargo (ex.: Analyst, Senior Analyst, Manager)
    age = Column(Integer)
    salary = Column(Float)  # Salário anual
    unit = Column(String)  # Unidade de negócios (ex.: IT, Finance, Marketing)
    leaves_used = Column(Integer)
    leaves_remaining = Column(Integer)
    ratings = Column(Float)
    past_exp = Column(Float)

# Criar uma conexão com o banco de dados SQLite
engine = create_engine('sqlite:///salarios.db')

# Criar as tabelas no banco de dados (se ainda não existirem)
Base.metadata.create_all(engine)

# Criar uma sessão
session = Session(engine)

try:
    # Criar a query usando o módulo select do SQLAlchemy
    stmt = select(
        Salaries.designation,
        func.min(Salaries.salary / 12).label('min_salary'),  # Salário mínimo mensal
        func.max(Salaries.salary / 12).label('max_salary'),  # Salário máximo mensal
        func.avg(Salaries.salary / 12).label('avg_salary')   # Salário médio mensal
    ).group_by(Salaries.designation)  # Agrupar por DESIGNATION

    # Executar a query usando a sessão
    results = session.execute(stmt)

    # Exibir os resultados
    print("\n=== Resultados da Consulta ===")
    for row in results:
        print(f"Designação: {row.designation}, "
              f"Mínimo: {row.min_salary:.2f}, "
              f"Máximo: {row.max_salary:.2f}, "
              f"Média: {row.avg_salary:.2f}")

finally:
    # Fechar a sessão após a execução
    session.close()


=== Resultados da Consulta ===
Designação: Analyst, Mínimo: 3333.42, Máximo: 4165.00, Média: 3751.68
Designação: Associate, Mínimo: 5846.17, Máximo: 8300.25, Média: 7266.92
Designação: Director, Mínimo: 17832.25, Máximo: 32342.67, Média: 23914.27
Designação: Manager, Mínimo: 8343.67, Máximo: 12407.50, Média: 10522.72
Designação: Senior Analyst, Mínimo: 4170.33, Máximo: 5830.50, Média: 4991.78
Designação: Senior Manager, Mínimo: 12614.42, Máximo: 16631.42, Média: 14888.69
